In [ ]:
# prep for mount
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#!python -m pip install opik

In [ ]:
#!python -m pip install wandb -Uq
#!python -m pip install ray[tune]
# !python -m pip install sigopt
# !python -m pip install optuna
!python -m pip install nltk python-Levenshtein
# !python -m pip install rouge-score
!python -m pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install comet_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.9/694.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 968.1/968.1 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.5/314.5 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 10.2 MB/s eta 0:00:00
  Created wheel for configobj: filename=configobj-5.0.9-py2.py3-none-any.whl size=35615 sha256=70346c57fe63c750af00c737432b3cd609189c1d84a071d939a802500d40b622
  Stored in directory: /root/.cache/pip/wheels/a1/6c/03/6c5e3cf1a6e4b9e2fc5c4409be4abc5a8268bd9c878739cb32
Successfully built configobj
  Attemp

In [ ]:
import comet_ml
import torch
import torch.nn as nn
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader, random_split
import time
import logging
import numpy as np
#import wandb
import random
import math
import os
import pandas as pd
import re
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity
import Levenshtein as lev
import nltk
#from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
#from rouge_score import rouge_scorer
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers.models.gpt2.modeling_gpt2 import GPT2Attention
from transformers import GPT2Config,  AutoModelForSeq2SeqLM
from peft import get_peft_model, IA3Config, TaskType
from transformers import TrainerCallback

In [ ]:
# os.environ["WANDB_API_KEY"] = "a5d8e3b6d2ef7d55f930ab72670aaa64e1a4198d"

# wandb.login() #enter this token: a5d8e3b6d2ef7d55f930ab72670aaa64e1a4198d
# #wandb.init(project="huggingface", entity="teamproject464-universit-t-des-saarlandes-saarland-unive")

In [ ]:
# import opik
# #kPnaiIq69hBXz7vVAWlDTtRed
# opik.configure(use_local=False)

In [ ]:

comet_ml.login(api_key='kPnaiIq69hBXz7vVAWlDTtRed', project_name='metaphor_detection', workspace='nadias')
experiment = comet_ml.Experiment()

COMET INFO: Valid Comet API Key saved in /content/drive/MyDrive/.comet.config (set COMET_CONFIG to change where it is saved).
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/nadias/metaphor-detection/b68bf9385b1c466884cc787bd7e0a13d



In [ ]:
!ls '/content/drive/MyDrive'

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


 annotation_met_new_version.csv   eng_lit_mihan.csv	      gpt-small-fr-finetuned
 annotation_nonmet.csv		  eng_met_mihan.csv	      gpt-small-frfr-finetuned
'Colab Notebooks'		  french_gpt_training.ipynb   lit_met.txt
'Copy of gpt_training.ipynb'	  german_gpt_training.ipynb   results
 df_met_final.txt		  gpt-finetuned
 eng_all_viet.csv		  gpt-small-eng-finetuned


In [ ]:

import pandas as pd
import re
# Preprocessing french data

def clean_output(text):
    # Remove commas, parentheses, and square brackets
    return re.sub(r'[\(\)\[\]\']', '', text)

def clean_output_special(text):
    if pd.isna(text):
        return text
    return re.sub(r'[\[\]\']' , '', text)

df_literal = pd.read_csv('/content/drive/MyDrive/lit_met.txt', sep='\t')
df_met = pd.read_csv('/content/drive/MyDrive/df_met_final.txt', sep='\t')

df_literal = df_literal.rename(columns={'Sentence': 'Input'})
df_met = df_met.rename(columns={'Sentence': 'Input'})

# clean output
df_literal['Output'] = df_literal['Output'].apply(clean_output_special)
df_met['Output'] = df_met['Output'].apply(clean_output_special)

df_literal['Input'] = df_literal['Input'].str.strip()
df_literal['Output'] = df_literal['Output'].str.strip()
df_literal = df_literal.dropna(subset=['Input', 'Output'])

df_met['Input'] = df_met['Input'].str.strip()
df_met['Output'] = df_met['Output'].str.strip()
df_met = df_met.dropna(subset=['Input', 'Output'])

fr_df_all = pd.concat([df_literal, df_met], ignore_index=True)
fr_df_all = fr_df_all.dropna()

# Put a placeholder for all the instances where there is no metaphor, as NAN cannot be procesed later
fr_df_all['Output'] = fr_df_all['Output'].replace("", " ")
# pad the ouptuts and ensure there is always a triple
def ensure_triple(data):
    result = []
    for item in data:
        #item = item.replace(",", "|")
        item_list = [x.strip() for x in item.split(",")]
        # If the item is a tuple or list, convert it to a list and check its length
        if len(item_list) < 3:
            item_list.append('')
            # If it has less than 3 elements, add 'nothing' to fill the missing slots
            while len(item_list) < 3:
                item_list.append("")

        item =" ".join(item_list)
        result.append(item)
    return result

fr_df_all['Output'] = ensure_triple(fr_df_all['Output'])
fr_df_all

,Input,Output
0,Des dispositions obligent les établissements f...,
1,Nous allons devoir trouver une solution à ce p...,
2,Je peux dire également que les Présidences fra...,
3,Pourquoi isoler le fondamentalisme religieux e...,
4,Je voudrais expliquer à Mme Izquierdo qu' il v...,
...,...,...
448,"Monsieur le Président , la confiance des citoy...",lunion européenne trouver réponse à problème
449,Je me demande si ceux qui ont multiplié leurs ...,eux multiplier intervention
450,Les drogues cultivées et produites en Afghanis...,drogue aboutir dans rue
451,Le rapport soulève néanmoins des questions imp...,prendre en compte question


In [ ]:
# Preprocessing english data

def clean_output(text):
    # Remove commas, parentheses, and square brackets
    return re.sub(r'[\(\)\[\]\']', '', text)

def clean_output_special(text):
    if pd.isna(text):
        return text
    return re.sub(r'[\[\]\']' , '', text)
df_literal = pd.read_csv('/content/drive/MyDrive/eng_lit_mihan.csv', delimiter=';', encoding='ISO-8859-1')


df_literal['Output'] = df_literal['Output'].apply(clean_output_special)
df_literal['Input'] = df_literal['Input'].str.strip()
df_literal['Output'] = df_literal['Output'].str.strip()
df_literal = df_literal.dropna(subset=['Input', 'Output'])
# size 350
# for index, row in df_cleaned.iterrows():
#     print(index)
#     print(row['Input'])
#     print(row['Output'])


# Output (Subject, Verb, Object For Active voices) and (Object, Verb, Subjet For Passive Voices)
df_met = pd.read_csv('/content/drive/MyDrive/eng_met_mihan.csv', delimiter=';', encoding='UTF-8')

df_met = df_met.rename(columns={'Output (Subject, Verb, Object For Active voices) and (Object, Verb, Subjet For Passive Voices)': 'Output'})
df_met['Output'] = df_met['Output'].apply(clean_output)
df_met['Input'] = df_met['Input'].str.strip()
df_met['Output'] = df_met['Output'].str.strip()
df_met = df_met.drop(columns=['Verb'])
df_met = df_met.dropna(subset=['Input', 'Output'])

# size 350
# Now Viet's
eng_all_viet = pd.read_csv('/content/drive/MyDrive/eng_all_viet.csv',delimiter=';', encoding='UTF-8' )
eng_all_viet = eng_all_viet.rename(columns={'Tuple SVO': 'Output'})
eng_all_viet = eng_all_viet.rename(columns={'Sentence': 'Input'})

# swap the columns to match the other dataset
cols = list(eng_all_viet.columns)

# Swap the first two elements
#cols[0], cols[1] = cols[1], cols[0]

# Reorder the DataFrame using the new column order
# why 399 ? 235 literal and 164 metaphores ? sure
#eng_all_viet = eng_all_viet[cols]

eng_all_viet['Output'] = eng_all_viet['Output'].apply(clean_output)
empty_count = 0
met_count = 0

for index, row in eng_all_viet.iterrows():
    if row['Output'] == '':
        empty_count += 1
    else:
        met_count += 1


eng_df_all = pd.concat([eng_all_viet, df_met, df_literal], ignore_index=True)


eng_df_all = eng_df_all.dropna()

# Put a placeholder for all the instances where there is no metaphor, as NAN cannot be procesed later
eng_df_all['Output'] = eng_df_all['Output'].replace("", "")
# pad the ouptuts and ensure there is always a triple
def ensure_triple(data):
    result = []
    for item in data:
        #item = item.replace(",", "|")
        item_list = [x.strip() for x in item.split(",")]
        # If the item is a tuple or list, convert it to a list and check its length
        if len(item_list) < 3:
            item_list.append('')
            # If it has less than 3 elements, add 'nothing' to fill the missing slots
            while len(item_list) < 3:
                item_list.append("")

        item =" ".join(item_list)
        result.append(item)
    return result

eng_df_all['Output'] = ensure_triple(eng_df_all['Output'])

eng_df_all.iloc[707]['Input']  # 415, 3, 700

'But he grasped the potential of putting the two together.'

In [ ]:
# Preprocessing german data

def clean_output(text):
    # Remove commas, parentheses, and square brackets
    return re.sub(r'[\(\)\[\]\']', '', text)

def clean_output_special(text):
    if pd.isna(text):
        return text
    return re.sub(r'[\[\]\']' , '', text)

df_literal = pd.read_csv('/content/drive/MyDrive/annotation_nonmet.csv', delimiter=';', header=None)
df_met = pd.read_csv('/content/drive/MyDrive/annotation_met_new_version.csv', delimiter=';', header=None)

df_literal.columns = ['Input', 'Output']
df_met.columns = ['Input', 'Output']
df_literal

# clean output


df_literal['Input'] = df_literal['Input'].str.strip()
df_literal['Output'] = df_literal['Output'].str.strip()
df_literal = df_literal.dropna(subset=['Input', 'Output'])

df_met['Input'] = df_met['Input'].str.strip()
df_met['Output'] = df_met['Output'].str.strip()
df_met = df_met.dropna(subset=['Input', 'Output'])


df_literal['Output'] = df_literal['Output'].apply(clean_output)
df_met['Output'] = df_met['Output'].apply(clean_output)
#reset indexes

df_met = df_met.reset_index(drop=True)
df_literal = df_literal.reset_index(drop=True)

ger_df_all = pd.concat([df_literal, df_met], ignore_index=True)
ger_df_all = ger_df_all.dropna()

# Put a placeholder for all the instances where there is no metaphor, as NAN cannot be procesed later

ger_df_all['Output'] = ger_df_all['Output'].replace("", " ")
# pad the ouptuts and ensure there is always a triple
def ensure_triple(data):
    result = []
    for item in data:
        #item = item.replace(",", "|")
        item_list = [x.strip() for x in item.split(",")]
        # If the item is a tuple or list, convert it to a list and check its length
        if len(item_list) < 3:
            item_list.append('')
            # If it has less than 3 elements, add 'nothing' to fill the missing slots
            while len(item_list) < 3:
                item_list.append("")

        item =" ".join(item_list)
        result.append(item)
    return result

ger_df_all['Output'] = ensure_triple(ger_df_all['Output'])
ger_df_all

<ipython-input-12-6a4c57aed967>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_literal['Output'] = df_literal['Output'].apply(clean_output)


,Input,Output
0,Die von der Stiftung Preußische Seehandlung ve...,
1,Die von der Budgetgemeindeversammlung beschlos...,
2,"Die werden sich an Sie erinnern, auch wenn Sie...",
3,Doch dann habe man den Sonderausschuss zu Beng...,
4,Doch dann wird Melanie mit Bens Vergangenheit ...,
...,...,...
341,Von rechten bis konservativen Politikern werde...,Angst schüren
342,Die Attentäter von Paris haben auf der Flucht ...,auf Spur bringen
343,"Um dieses Ziel zu erreichen, ist Einsatz gefra...",fahren an Limit
344,Auch wenn man schöne Zeiten wie den Aufstieg o...,Lücke schließen


In [ ]:
# Optimizersss
#1. Optuna

def optuna_hp_space(trial):

    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64, 128]),
    }

#2. SigOpt

def sigopt_hp_space(trial):

    return [
        {"bounds": {"min": 1e-6, "max": 1e-4}, "name": "learning_rate", "type": "double"},
        {
            "categorical_values": ["16", "32", "64", "128"],
            "name": "per_device_train_batch_size",
            "type": "categorical",
        },
    ]

#3. raytune

def ray_hp_space(trial):
    return {
        "learning_rate": tune.loguniform(1e-6, 1e-4),
        "per_device_train_batch_size": tune.choice([16, 32, 64, 128]),
    }

#4.Wandb
def wandb_hp_space(trial):
    return {
        "method": "random",
        "metric": {"name": "objective", "goal": "minimize"},
        "parameters": {
            "learning_rate": {"distribution": "uniform", "min": 1e-6, "max": 1e-4},
            "per_device_train_batch_size": {"values": [16, 32, 64, 128]},
        },
    }

In [ ]:
# # include new metrics in class TRAINER

# class CustomTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         # Get model predictions
#         outputs = model(**inputs)
#         logits = outputs.get('logits')
#         labels = inputs.get('labels')

#         # Convert logits to predicted tokens
#         predictions = torch.argmax(logits, dim=-1)

#         # Compute standard cross-entropy loss (you can keep this for token-level guidance)
#         loss_fct = torch.nn.CrossEntropyLoss()
#         loss = loss_fct(logits.view(-1, self.model.config.vocab_size), labels.view(-1))

#         # Convert predictions and labels to text (for custom metrics)
#         pred_texts = [tokenizer.decode(pred, skip_special_tokens=True).strip() for pred in predictions]
#         label_texts = [tokenizer.decode(label, skip_special_tokens=True).strip() for label in labels]

#         # 1. Cosine Similarity Metric
#         cos_loss = 0.0
#         for pred_text, label_text in zip(pred_texts, label_texts):
#             if pred_text and label_text: # check that pred and label are non-empty
#               pred_vec = tokenizer.encode(pred_text, return_tensors='pt')
#               label_vec = tokenizer.encode(label_text, return_tensors='pt')
#               if pred_vec.shape[1] > 0 and label_vec.shape[1] > 0: # check that arrays are non-empty
#                 cos_sim = cosine_similarity(pred_vec.detach().numpy(), label_vec.detach().numpy())
#                 cos_loss += 1 - cos_sim.mean()  # Convert similarity to distance
#               else:
#                 # Handle cases where vectors are empty, possibly by assigning a default loss
#                 cos_loss += 1 # Assign highest possible loss (1) if vectors are empty
#             else:
#               cos_loss += 1 #Assign highest possible loss if predictions or labels are empty

#         # Normalize the cosine loss by batch size
#         cos_loss /= len(pred_texts)

#         # 2. Levenshtein Distance Metric
#         lev_loss = 0.0
#         for pred_text, label_text in zip(pred_texts, label_texts):
#             lev_distance = Levenshtein.distance(pred_text, label_text)
#             max_len = max(len(pred_text), len(label_text))
#             lev_loss += lev_distance / max_len  # Normalize Levenshtein distance by the max length

#         # Normalize Levenshtein loss by batch size
#         lev_loss /= len(pred_texts)

#         # Combine the original cross-entropy loss with custom metrics
#         combined_loss = loss + cos_loss + lev_loss

#         return (combined_loss, outputs) if return_outputs else combined_loss

In [ ]:
# import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
def levenshtein_metric(predictions, references):
    distances = []
    for pred, ref in zip(predictions, references):
        pred_text = tokenizer.decode(pred, skip_special_tokens=True)
        ref_text = tokenizer.decode(ref, skip_special_tokens=True)
        # Compute the normalized Levenshtein distance
        distance = lev.distance(pred_text, ref_text) / max(len(pred_text), len(ref_text))
        distances.append(1 - distance)  # Invert it to represent similarity
    return np.mean(distances)

In [ ]:
def token_level_metrics(predictions, references):
    precision_list, recall_list, f1_list = [], [], []
    for pred, ref in zip(predictions, references):
        pred_ids = pred.flatten()  # Flatten the tensor if needed
        ref_ids = ref.flatten()

        # Ignore padding tokens (or other special tokens if defined)
        mask = ref_ids != tokenizer.pad_token_id

        # Compute precision, recall, f1
        precision = precision_score(ref_ids[mask], pred_ids[mask], average='micro')
        recall = recall_score(ref_ids[mask], pred_ids[mask], average='micro')
        f1 = f1_score(ref_ids[mask], pred_ids[mask], average='micro')

        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

    return np.mean(precision_list), np.mean(recall_list), np.mean(f1_list)

In [ ]:
nltk.download('punkt')

# def bleu_score_metric(predictions, references):
#     bleu_scores = []
#     for pred, ref in zip(predictions, references):
#         pred_tokens = tokenizer.decode(pred, skip_special_tokens=True).split()
#         ref_tokens = tokenizer.decode(ref, skip_special_tokens=True).split()

#         # Calculate the BLEU score for each pair of sentences
#         bleu_scores.append(sentence_bleu([ref_tokens], pred_tokens))

#     return np.mean(bleu_scores)
def bleu_score_metric(predictions, references):
    bleu_scores = []
    smoothing_function = SmoothingFunction().method4  # Apply smoothing to handle missing n-grams
    for pred, ref in zip(predictions, references):
        pred_tokens = tokenizer.decode(pred, skip_special_tokens=True).split()
        ref_tokens = tokenizer.decode(ref, skip_special_tokens=True).split()

        # Calculate the BLEU score for each pair of sentences with smoothing
        bleu_scores.append(sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smoothing_function))

    return np.mean(bleu_scores)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:

def calculate_rouge(reference, hypothesis):
    # Initialize ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Compute ROUGE scores
    scores = scorer.score(reference, hypothesis)
    return scores

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    levenshtein_sim = levenshtein_metric(predictions, labels)
    precision, recall, f1 = token_level_metrics(predictions, labels)
    return {
        'levenshtein_similarity': levenshtein_sim,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [ ]:
# # Hypertune with IA3
# class IA3Layer(nn.Module):
#     def __init__(self, hidden_dim):
#         super(IA3Layer, self).__init__()
#         # Learnable scaling vectors for keys, queries, and values
#         self.ia3_key = nn.Parameter(torch.ones(hidden_dim))
#         self.ia3_query = nn.Parameter(torch.ones(hidden_dim))
#         self.ia3_value = nn.Parameter(torch.ones(hidden_dim))

#     def forward(self, query, key, value):
#         # Apply input-aware scaling to keys, queries, and values

#         print(f"Query shape before IA3: {query.shape}", flush=True)
#         print(f"Key shape before IA3: {key.shape}",  flush=True)
#         print(f"Value shape before IA3: {value.shape}",  flush=True)

#         query = query * self.ia3_query
#         key = key * self.ia3_key
#         value = value * self.ia3_value

#         print(f"Query shape after IA3: {query.shape}",  flush=True)
#         print(f"Key shape after IA3: {key.shape}",  flush=True)
#         print(f"Value shape after IA3: {value.shape}",  flush=True)
#         return query, key, value

In [ ]:

# class GPT2AttentionWithIA3(GPT2Attention):
#     def __init__(self, config):
#         super().__init__(config)
#         # Initialize IA3 scaling layers
#         self.ia3 = IA3Layer(config.hidden_size)

#     def forward(self, hidden_states, layer_past=None, attention_mask=None, head_mask=None, use_cache=False, output_attentions=False):
#         # Perform the standard GPT-2 attention mechanism
#         print("1",  flush=True)
#         query, key, value = self._attn(hidden_states, attention_mask, head_mask, use_cache, output_attentions)
#         print("2",  flush=True)
#         # Apply IA3 scaling
#         query, key, value = self.ia3(query, key, value)
#         print("3",  flush=True)
#         # Proceed with the rest of the attention mechanism (unchanged)
#         return super().forward(hidden_states, layer_past, attention_mask, head_mask, use_cache, output_attentions)

In [ ]:
# class GPT2WithIA3(GPT2LMHeadModel):
#     def __init__(self, config):
#         super().__init__(config)
#         # Replace the attention layers with IA3-enhanced layers
#         for layer in self.transformer.h:
#             layer.attn = GPT2AttentionWithIA3(config)

# # Load the pre-trained GPT-2 model and replace the attention layers
# config = GPT2Config.from_pretrained("gpt2")
# model = GPT2WithIA3(config)

In [ ]:
model_name="antoiloui/belgpt2"

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
#config = GPT2Config.from_pretrained("gpt2")
#model = GPT2WithIA3(config)

# Add the padding token to GPT-2's tokenizer (optional, but useful)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Hook to capture layer activations
def get_layer_weights_hook(module, input, output):
    # Save the output (weights after decoding) from the layer
    module.layer_outputs = output.detach().cpu().numpy()

# Attach the hook to a specific layer you want to probe (e.g., the decoder layer)
model.lm_head.register_forward_hook(get_layer_weights_hook)
class TextDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=32, max_length_out = 32):
        self.inputs = df['Input'].tolist()
        self.outputs = df['Output'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.max_length_out= max_length_out

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        output_text = self.outputs[idx]

        tokenized_output = self.tokenizer(output_text, truncation=True, return_tensors='pt', padding='max_length', max_length=self.max_length_out)
        tokenized_input = self.tokenizer(input_text, truncation=True, return_tensors='pt', padding='max_length', max_length=self.max_length)
        #print(tokenized_input['input_ids'].shape, tokenized_output['input_ids'].shape)

        # create Mask for empty and delimiter
        empty_token_id = self.tokenizer.convert_tokens_to_ids('<EMPTY>')
        output_mask = (output_ids != empty_token_id).long()
        # resize the model after introducing special tokens # and |
        model.resize_token_embeddings(len(tokenizer))
        # Return input_ids and attention_mask for training, no labels

        # Input IDs and attention mask
        input_ids = tokenized_input['input_ids'].squeeze()  # shape: (max_length)
        attention_mask = tokenized_input['attention_mask'].squeeze()  # shape: (max_length)
        output_ids = tokenized_output['input_ids'].squeeze()  # shape: (max_length)

        # Return input_ids and attention_mask for training, no labels
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': output_ids,
            'output_mask': output_mask
        }

class MaskedTextDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=32, max_length_out = 32):
        self.inputs = df['Input'].tolist()
        self.outputs = df['Output'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.max_length_out= max_length_out
        self.stored_input_ids = []  # Store the input IDs here
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        output_text = self.outputs[idx]

        tokenized_output = self.tokenizer(output_text, truncation=True, return_tensors='pt', padding='max_length', max_length=self.max_length)
        tokenized_input = self.tokenizer(input_text, truncation=True, return_tensors='pt', padding='max_length', max_length=self.max_length)
        #print(tokenized_input['input_ids'].shape, tokenized_output['input_ids'].shape)
        # Input IDs and attention mask
        input_ids = tokenized_input['input_ids'].squeeze()  # shape: (max_length)
        attention_mask = tokenized_input['attention_mask'].squeeze()  # shape: (max_length)
        output_ids = tokenized_output['input_ids'].squeeze()  # shape: (max_length)

        # create Mask for empty and delimiter
        #hash_token_id = self.tokenizer.convert_tokens_to_ids('#')
        #delimiter_token_id = self.tokenizer.convert_tokens_to_ids('|')

        #output_mask = (output_ids != hash_token_id).long() & (output_ids != delimiter_token_id).long()
        self.stored_input_ids.append(input_ids.tolist())  # Store the input IDs
        # Return input_ids and attention_mask for training, no labels
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': output_ids
        }

class TextDatasetWithPrompts(Dataset):
    def __init__(self, df, tokenizer, max_length=64, max_length_out=64):
        self.inputs = df['Input'].tolist()
        self.outputs = df['Output'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.max_length_out = max_length_out

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = f"Input: {self.inputs[idx]}\nOutput:"
        output_text = self.outputs[idx]

        tokenized_input = self.tokenizer(
            input_text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt'
        )
        tokenized_output = self.tokenizer(
            output_text, truncation=True, padding='max_length', max_length=self.max_length_out, return_tensors='pt'
        )

        input_ids = tokenized_input['input_ids'].squeeze()
        attention_mask = tokenized_input['attention_mask'].squeeze()
        output_ids = tokenized_output['input_ids'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': output_ids
        }

# Load data into the custom dataset
My_dataset = MaskedTextDataset(df=fr_df_all, tokenizer=tokenizer)
# Split into training and test sets
split = 0.8
train_eval_size = int(split * len(My_dataset))
test_size = len(My_dataset) - train_eval_size
train_eval_data, test_data = random_split(My_dataset, [train_eval_size, test_size])

train_size = int(split * len(train_eval_data))
eval_size = len(train_eval_data) - train_size
training_data, eval_data = random_split(train_eval_data, [train_size, eval_size])

# Define DataLoaders
batch_size = 64
epochs = 50
# dataloader_train = DataLoader(training_data, batch_size=batch_size, shuffle=True)
# dataloader_eval = DataLoader(eval_data, batch_size=batch_size, shuffle=True)
# dataloader_test = DataLoader(test_data, batch_size=batch_size, shuffle=True)
######
# masking outside
#special_tokens_dict = {'additional_special_tokens': ['#']}
#tokenizer.add_special_tokens(special_tokens_dict)

class LogInputIDsCallback(TrainerCallback):
    def __init__(self, experiment, tokenizer, My_dataset):
        self.experiment = experiment
        self.tokenizer = tokenizer
        self.dataset = My_dataset
    def on_log(self, args, state, control, **kwargs):
        step = state.global_step
        print(step)
        # Get the corresponding input_ids from the dataset
        input_ids = self.dataset.stored_input_ids[-step:]
        print(input_ids)
        if input_ids is not None:
              for ids in input_ids:
                  tokens = self.tokenizer.convert_ids_to_tokens(ids)
                  tokens = self.tokenizer.decode(tokens)
                  token_str = " ".join(tokens)
                  print(f"Logging tokens for step {step}: {token_str}")

                  # Log to Comet
                  self.experiment.log_text(f"Tokens: {token_str}", metadata={"step": step})
######
# Define training arguments
training_args = TrainingArguments(
    logging_steps=50,
    output_dir='./results-small',
    learning_rate = 5e-4,
    weight_decay = 0.01,
    gradient_accumulation_steps=3,
    overwrite_output_dir=True,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_strategy="epoch",
    report_to="comet_ml"  # This enables reporting to Comet
)

# Model Init for hyperparam optimization


# model_args = {
#     "model": model_name,
#     "from_tf": False,
#     "config": AutoConfig.from_pretrained(model_name),
#     "cache_dir": None, # You can specify a cache directory if needed
#     "revision": None, # You can specify a model revision if needed
#     "token": True # Set to True if using an authentication token
# }

# def model_init(trial):
#     model = GPT2LMHeadModel.from_pretrained(
#         model_name,
#         ignore_mismatched_sizes=True,
#         from_tf=bool(".ckpt" in model_name),
#         config= AutoConfig.from_pretrained(model_name),
#         cache_dir=None,
#         revision=None,
#         token=True,
#     )
#     model.config.pad_token_id = tokenizer.eos_token_id
#     return model

#IA3 init config
peft_config = IA3Config(
    task_type=TaskType.CAUSAL_LM,  # Task type (change if necessary)
    target_modules=["attn.c_attn","attn.c_proj", "mlp.c_proj","mlp.c_fc"],  # Attention block layers to apply IA3
    feedforward_modules=["mlp.c_fc"],  # Apply to feedforward layers
    modules_to_save=["lm_head"],  # List of trainable modules to save
)
model.resize_token_embeddings(len(tokenizer))
print(model)
# Wrap the model using the PEFT IA3 method
model = get_peft_model(model, peft_config)

# Optional: Print trainable parameters
model.print_trainable_parameters()

#Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_data,
    eval_dataset=eval_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50258, bias=False)
)
trainable params: 38,653,440 || all params: 163,094,016 || trainable%: 23.7001


/usr/local/lib/python3.10/dist-packages/peft/tuners/ia3/model.py:134: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
# #Initialize the Trainer with wandb stuff
# trainer = Trainer(
#     model_init=model_init, # This is for the model init for the hyperparm optim
#     model=model,
#     args=training_args,
#     train_dataset=training_data,
#     eval_dataset=eval_data
# )
# trainer = CustomTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=training_data,
#     eval_dataset=eval_data,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics  # Optionally include custom metrics
# )

In [ ]:
#
# ## Get best hyperparams
# best_trials = trainer.hyperparameter_search(
#     direction=["minimize", "maximize"],
#     backend="optuna",
#     hp_space=optuna_hp_space,
#     n_trials=10
# )

In [ ]:

# Start finetuning
start_time = time.time()
print('Training starts')
trainer.train()
model.save_pretrained("/content/drive/MyDrive/gpt-small-frfr-finetuned")
#model.save_weights("/content/drive/MyDrive/weights.h5")
print('Training done')
end_time = time.time()
experiment.end()

COMET INFO: An experiment with the same configuration options is already running and will be reused.


Training starts


Epoch,Training Loss,Validation Loss,Levenshtein Similarity,Precision,Recall,F1
0,No log,15.819966,0.050115,0.000000,0.000000,0.000000
1,No log,14.990126,0.050000,0.000000,0.000000,0.000000
3,No log,13.906813,0.050437,0.000000,0.000000,0.000000
4,No log,13.271584,0.050417,0.000000,0.000000,0.000000
6,No log,12.377707,0.050706,0.000000,0.000000,0.000000
7,No log,11.804815,0.051631,0.000000,0.000000,0.000000
9,No log,11.004110,0.051436,0.000000,0.000000,0.000000
10,No log,10.544038,0.051099,0.000000,0.000000,0.000000
12,No log,9.878005,0.051488,0.000000,0.000000,0.000000
13,No log,9.469771,0.051361,0.000000,0.000000,0.000000


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Training done


In [ ]:
fine_tuning_time = end_time - start_time
print(f"Training completed in {fine_tuning_time:.2f} seconds.")
trainer.evaluate()

Training completed in 911.11 seconds.


{'eval_loss': 6.971639156341553,
 'eval_levenshtein_similarity': 0.05345326354677708,
 'eval_precision': 0.00684931506849315,
 'eval_recall': 0.00684931506849315,
 'eval_f1': 0.00684931506849315,
 'eval_runtime': 3.7258,
 'eval_samples_per_second': 19.593,
 'eval_steps_per_second': 0.537,
 'epoch': 30.0}

In [ ]:
predictions, labels, metrics = trainer.predict(test_dataset=test_data)

In [ ]:
predicted_token_ids = np.argmax(predictions, axis=-1)

# Step 2: Decode the predicted token IDs to text
decoded_predictions = [tokenizer.decode(pred_seq, skip_special_tokens=True) for pred_seq in predicted_token_ids]
decoded_predictions

["ée que,, des des,� nous que nous l marché en commun nos énergie � notre efforts �' actionsergie �       ",
 ' les garde de de, Président, exercice du de ne pas être de temps de le position exemple sujet à bras lele-corps.Je la la la la',
 " été un'  la bases de' une nouveau de retraite de, son.        que que que que que,,,,",
 " été trouvernons à. et nous' a pas eu' excuseclusments les mal. la il la à la être communauté de' êtreayer de vaincre",
 " les,, qui laquierdo que' en est mieux mieux être les nombre de TVA que la' économie que de stimuler les' environnement des nous le le",
 " été, aborder pour concert pour les partenaires pour que une les même chose des la' œuvrero la' austérité. le vies.dépenddépenddépenddépenddépend",
 'ons-vous nous à la question? de je ne lave? études?................',
 " été,1 année, cette d d' autres,, à droits de, est' ont pas une.e ce ce l l l l l l l",
 " monde de la de' Etat, en la' influenceé de le vie. affaiblir années à venir.,,,,,,, et et et 

In [ ]:
decoded_labels = [tokenizer.decode(label_seq, skip_special_tokens=True) for label_seq in labels]
decoded_labels

['  ',
 'nous prendre problème à bras le corps',
 '  ',
 'Turquie emprunter voie',
 '  ',
 '  ',
 'je poursuivre remarque',
 'directive toucher à question',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 ' stimuler de activité',
 '  ',
 '  ',
 '  ',
 'gouvernement émettre doute',
 '  ',
 'confiance se retrouver en lambeaux',
 'nous sattaquer à question',
 'abandonner proposition ',
 '  ',
 ' aborder problème',
 'collègue répondre à inquiétude',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 ' résorber inégalité',
 'je clôturer affaire',
 'je soulever point',
 '  ',
 'je décrocher emploi',
 '  ',
 'il lutter contre problème',
 'nous affronter défi',
 '  ',
 '  ',
 'nous trouver moyen',
 'autorité saisir concept',
 ' satteler à tâche',
 'je préciser point',
 '  ',
 '  ',
 '  ',
 '  ',
 'Rejeter être source de confusion',
 '  ',
 'il se pencher sur problème',
 '  ',
 'le Premier ministre trouver terrain dentente',
 '  ',
 '  ',
 '  ',
 '  ',
 'aspect être aborder ',
 '  ',
 'problème être 

In [ ]:
# human comparison:
for label,prediction in zip(decoded_labels,decoded_predictions):
  print(f'Label:{label}, Prediction:{prediction}')

Label:  , Prediction:ée que,, des des,� nous que nous l marché en commun nos énergie � notre efforts �' actionsergie �       
Label:nous prendre problème à bras le corps, Prediction: les garde de de, Président, exercice du de ne pas être de temps de le position exemple sujet à bras lele-corps.Je la la la la
Label:  , Prediction: été un'  la bases de' une nouveau de retraite de, son.        que que que que que,,,,
Label:Turquie emprunter voie, Prediction: été trouvernons à. et nous' a pas eu' excuseclusments les mal. la il la à la être communauté de' êtreayer de vaincre
Label:  , Prediction: les,, qui laquierdo que' en est mieux mieux être les nombre de TVA que la' économie que de stimuler les' environnement des nous le le
Label:  , Prediction: été, aborder pour concert pour les partenaires pour que une les même chose des la' œuvrero la' austérité. le vies.dépenddépenddépenddépenddépend
Label:je poursuivre remarque, Prediction:ons-vous nous à la question? de je ne lave? études?.........

In [ ]:
# Transfer learning script

# Load data into the custom dataset
My_dataset_eng = MaskedTextDataset(df=eng_df_all, tokenizer=tokenizer)
# Split into training and test sets
split = 0.8
train_eval_size_eng = int(split * len(My_dataset_eng))
test_size_eng = len(My_dataset_eng) - train_eval_size_eng
train_eval_data_eng, test_data_eng = random_split(My_dataset_eng, [train_eval_size_eng, test_size_eng])

My_dataset_ger = MaskedTextDataset(df=ger_df_all, tokenizer=tokenizer)
train_eval_size_ger = int(split * len(My_dataset_ger))
test_size_ger = len(My_dataset_ger) - train_eval_size_ger
train_eval_data_ger, test_data_ger = random_split(My_dataset_ger, [train_eval_size_ger, test_size_ger])



In [ ]:
predictions, labels, metrics = trainer.predict(test_dataset=test_data_eng)

In [ ]:
predicted_token_ids = np.argmax(predictions, axis=-1)

# Step 2: Decode the predicted token IDs to text
decoded_predictions = [tokenizer.decode(pred_seq, skip_special_tokens=True) for pred_seq in predicted_token_ids]
decoded_labels = [tokenizer.decode(label_seq, skip_special_tokens=True) for label_seq in labels]
# human comparison:
for label,prediction in zip(decoded_labels,decoded_predictions):
  print(f'Label:{label}, Prediction:{prediction}')




Label:we kill off program, Prediction:onstonter  'yfic, theinding the a whos. havegue' t theing onÂ la la la la la la la'
Label:  , Prediction: étéons the year the place leadinary, of thetermination to the de who wasaped the a car, in in a way dayy. the
Label:  , Prediction: mondepartment s,'' s oning,000-1 victor000 a 3erson' run. the firstth inning.000 the notppedta
Label:  , Prediction: est, ,lly, to doach awayzybt,P la la la la la la la''''''''
Label:  , Prediction:ons, qui unérantaire,aceler, who wasdles the a ndom'er. of the mostviouscks hales ofver.d
Label:newsletter attack this, Prediction:ons de close of tolytokette.ction. help the.T l l l l l l l l l l livre    
Label:water be poured , Prediction:ons, been ared by the gr since the othervisd been aed.. the ownffic. said s s saide'.'les les
Label:  , Prediction:ésing the governmentagues have notamed tchedver to doy the, he were not mores liager to have him ownatherts. to they
Label:  , Prediction:ée.e.ke qui formerress,mer, isd

In [ ]:
metrics

{'test_loss': 12.64858627319336,
 'test_levenshtein_similarity': 0.06673972615206723,
 'test_precision': 0.003943374633029806,
 'test_recall': 0.003943374633029806,
 'test_f1': 0.003943374633029806,
 'test_runtime': 8.7081,
 'test_samples_per_second': 19.981,
 'test_steps_per_second': 0.345}

In [ ]:
predictions, labels, metrics = trainer.predict(test_dataset=test_data_ger)

In [ ]:
predicted_token_ids = np.argmax(predictions, axis=-1)

# Step 2: Decode the predicted token IDs to text
decoded_predictions = [tokenizer.decode(pred_seq, skip_special_tokens=True) for pred_seq in predicted_token_ids]
decoded_labels = [tokenizer.decode(label_seq, skip_special_tokens=True) for label_seq in labels]
# human comparison:
for label,prediction in zip(decoded_labels,decoded_predictions):
  print(f'Label:{label}, Prediction:{prediction}')


Label:  , Prediction:éetaer Jahem,chlftigt,,ch inerorsri,lassenreoche,, ität.ürich
Label:  , Prediction:ons,raurde i e d desumächst.ur,gangendenen dk inersmle unden.
Label:  , Prediction:és,ll,,ilien,ben,ur, eil,it,ä,irglich, .Soliner.undse.te
Label:sich Vorstellung erstrecken, Prediction:ée detver snischen Kachchritts.iren..terre orschung. e isets des der.
Label:  , Prediction:onsgo faiten,achlfer,,iterel,.ückber,achhrung. ..b.. auatser
Label:Fuß fassen , Prediction: hui     ',,,  enil,er endlichen, tenzeu..  ersten "
Label:  , Prediction:onsine,,ur, ersm e en Jah. undtraenerstteinen.achich.rscheinlich.. 
Label:Lücke schließen , Prediction: monde,ir es,wne,,,der fchg,der  - kt.tenach. der einer 
Label:  , Prediction:onsel,lteesetzichert,enn.m.ch,.lt. ir es ach, derm "deren,
Label:  , Prediction:onsgoen,,ach  "idenifenen schon.osenbauer. iemir Ktin htum Jahillionnern s
Label:  , Prediction:éeée.umtzt,el es,  egenes,ßnah,geen�ffonomische achmmausK 
Label:  , Prediction:ons,,,trst, n.jte

In [ ]:
metrics

{'test_loss': 1.691941738128662,
 'test_levenshtein_similarity': 0.26926719367565816,
 'test_precision': 0.2642857142857143,
 'test_recall': 0.2642857142857143,
 'test_f1': 0.2642857142857143,
 'test_runtime': 3.1693,
 'test_samples_per_second': 22.087,
 'test_steps_per_second': 0.631}